# 讀出所有方志階層檔，並存於 Book obj 中

In [1]:
filepath = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC'

In [2]:
import os

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(filepath):
    for file in f:
        if '._' not in file and '.xml' in file and file != 'bookId.xml':
            files.append(os.path.join(r, file))

print(len(files))
print(files[0:5])

3998
['/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/35392.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/97486.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/35386.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/92850.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/36842.xml']


In [3]:
import xmltodict, json

Books = {}

def singleChapter(INchapter, INparent, chapterTitles):
    _title = INchapter['title']
    
    _from = ""
    if '@from' in INchapter:
        _from = INchapter['@from']
    
    _to = ""
    if '@to' in INchapter:
        _to = INchapter['@to']
        
    _level = ""
    if '@level' in INchapter:
        _level = INchapter['@level']
        
    _n = ""
    if '@n' in INchapter:
        _n = INchapter['@n']

    # 'parent' = [sub]
#     print(INparent, _title)
    if INparent not in chapterTitles:
        chapterTitles[INparent] = [_title]
    elif _title not in chapterTitles[INparent]:
        chapterTitles[INparent].append(_title)
        
    # [sub] = [parent]
#     if _title not in chapterTitles:
#         chapterTitles[_title] = [INparent]
#     elif INparent not in chapterTitles[_title]:
#         chapterTitles[_title].append(INparent)
    
def section(INarray, INtitle, chapterTitles):
#     print("section::", INtitle, type(INarray), INarray)
    if type(INarray) == dict:
        singleChapter(INarray, INtitle, chapterTitles)
        if 'section' in INarray:
            section(INarray['section'], INarray['title'], chapterTitles) 
        
    elif type(INarray) == list:    
        for ch in INarray:
            section(ch, INtitle, chapterTitles)
            if 'section' in ch:
                section(ch['section'], ch['title'], chapterTitles)
                
    else:
        print(INarray)
        print("ERROR:", type(INarray))
        
    return chapterTitles

# file: [35393].xml
def recursiveSection(INdata):
    chapterTitles = {}
    book = INdata['title']

    if 'section' in INdata:
        titleObj = section(INdata['section'], book, chapterTitles)
    return titleObj

def parseFile(file):
    with open(file) as fp:
        xmlStr = fp.read()
        convertedDict = xmltodict.parse(xmlStr);
        jsonStr = json.dumps(convertedDict, ensure_ascii=False)
        jsonObj = json.loads(jsonStr)

#         print(jsonObj)
        book = jsonObj['book']

        titleObj = recursiveSection(book)
#         print(titleObj)
    
        rstart = file.rfind('/')
        rend = file.rfind('.')
        # file: [35393].xml
        bid = file[rstart+1:rend]
        
        bname = list(titleObj.keys())[0]         
        Books[bid] = {'book': bname, 'data': titleObj}
    
# print(files[0])
# parseFile(files[0])
# print(Books)
# parseFile("/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/97725.xml")

In [4]:
# root = AnyNode(id="root")
for file in files:
    parseFile(file)
print(len(Books))
# parseFile(files[0])

3998


In [5]:
# the first 
# print(Books[list(Books.keys())[0]])
# print(Books['17106'])
# print(Books['97725'])
print(Books['35282'])


{'book': '涇縣續志', 'data': {'涇縣續志': ['山澤附（闕）', '節婦（闕）', '涇縣續志序', '涇縣續志纂修姓氏', '涇縣續志目錄', '卷一', '卷二', '卷三', '卷四', '卷五', '卷六', '卷七', '卷八', '卷九'], '卷一': ['學校', '書院', '壇廟', '公館', '橋梁', '津渡', '古蹟', '寺觀'], '書院': ['學田'], '古蹟': ['冡墓'], '卷二': ['食貨', '職官表', '選舉表'], '食貨': ['蠲賑', '恩賚', '户口', '積貯'], '選舉表': ['例仕', '吏仕', '武選舉', '武職', '贈封', '歷襲'], '卷三': ['人物'], '人物': ['名臣', '宦業', '忠節', '孝友', '儒林', '文苑', '武功', '懿行', '尚義', '五世同堂', '百歳', '隱逸', '藝術', '寓賢'], '卷四': ['人物'], '卷五': ['人物'], '卷六': ['人物'], '卷七': ['列女'], '列女': ['已旌', '烈婦：已旌', '貞女', '壽婦', '節婦：已故待旌', '烈婦', '孝女', '現年節婦'], '卷八': ['列女'], '卷九': ['藝文', '災祥', '附刻煙石嶺禁碑并\uedce字']}}


# 哪幾本方志階層不用修正 

In [6]:
import sys

def showLevelGap(level):
    x = ""
    for idx in range(0, level):
        x += "==="
    return x
    
def showSubChapter(subs, data, level, uppers):
    if level > 10:
        sys.exit()

    for s in subs:
        prefix = showLevelGap(level)
        print(prefix, s)
        if s not in data:
            continue

        if level == 1:
#             print("LEVEL 1::", uppers)
            uppers = []
        else:
            if s in uppers:
                continue
            else:
                uppers.append(s)
            
        nxtSubs = data[s]
        showSubChapter(nxtSubs, data, level+1, uppers)
    
# read one gazetters
def showGazetter(bid):
    if bid not in Books:
        print("ERROR!!", bid)
        return
    gazetteer = Books[bid]
    title = gazetteer['book']
    print(title)

    data = gazetteer['data']
#     print(data)
    
    firstLevels = data[title]
    uppers = []
    showSubChapter(firstLevels, data, 1, uppers)
            
# showGazetter('35392')
# showGazetter('17106')
# showGazetter('97725')
showGazetter('35282')


涇縣續志
=== 山澤附（闕）
=== 節婦（闕）
=== 涇縣續志序
=== 涇縣續志纂修姓氏
=== 涇縣續志目錄
=== 卷一
====== 學校
====== 書院
========= 學田
====== 壇廟
====== 公館
====== 橋梁
====== 津渡
====== 古蹟
========= 冡墓
====== 寺觀
=== 卷二
====== 食貨
========= 蠲賑
========= 恩賚
========= 户口
========= 積貯
====== 職官表
====== 選舉表
========= 例仕
========= 吏仕
========= 武選舉
========= 武職
========= 贈封
========= 歷襲
=== 卷三
====== 人物
========= 名臣
========= 宦業
========= 忠節
========= 孝友
========= 儒林
========= 文苑
========= 武功
========= 懿行
========= 尚義
========= 五世同堂
========= 百歳
========= 隱逸
========= 藝術
========= 寓賢
=== 卷四
====== 人物
========= 名臣
========= 宦業
========= 忠節
========= 孝友
========= 儒林
========= 文苑
========= 武功
========= 懿行
========= 尚義
========= 五世同堂
========= 百歳
========= 隱逸
========= 藝術
========= 寓賢
=== 卷五
====== 人物
========= 名臣
========= 宦業
========= 忠節
========= 孝友
========= 儒林
========= 文苑
========= 武功
========= 懿行
========= 尚義
========= 五世同堂
========= 百歳
========= 隱逸
========= 藝術
========= 寓賢
=== 卷六
====== 人物
========= 名臣
========= 宦業
========= 忠

In [7]:
import sys
    
def checkSubChapter(subs, data, level, uppers):
    if level > 10:
        sys.exit()

    for s in subs:
        if type(s) != str:
            continue
        if len(s) > 3:
            return False
        if s not in data:
            continue
            
        if level == 1:
            uppers = []
        else:
            if s in uppers:
                continue
            else:
                uppers.append(s)

        nxtSubs = data[s]
        if checkSubChapter(nxtSubs, data, level+1, uppers) == False:
            return False
    return True

# check gazetter's chapter categories are correct or not
def checkGazetter(bid):
    if bid not in Books:
        print("ERROR!!", bid)
        return False
    gazetteer = Books[bid]
    title = gazetteer['book']

    data = gazetteer['data']
    firstLevels = data[title]
    return checkSubChapter(firstLevels, data, 1, [])

# checkGazetter('36789')
# checkGazetter('98012')
checkGazetter('97725')


False

In [8]:
easyGazetters = []
for b in Books:
    if checkGazetter(b):
        easyGazetters.append(b)
print(len(easyGazetters))
print(easyGazetters)

17
['98145', '97953', '97622', '94355', '97017', '93658', '93659', '94291', '92903', '37039', '71417', '93555', '98144', '93051', '16649', '93602', '97806']


In [9]:
showGazetter('97806')

南川縣志
=== 沿革
=== 星野
=== 形勝
=== 城池
=== 公署
=== 學校
=== 風俗
=== 戶口
=== 田賦
=== 山川
=== 橋樑
=== 祠觀
=== 古蹟
=== 秩官
=== 節烈
=== 名宦
=== 鄉賢
=== 人物


# Write chapters to file

In [10]:
# allChapters = []

# for key in chapterTitles:
#     if key not in allChapters:
#         allChapters.append(key)
#     for c in chapterTitles[key]:
#         if c not in allChapters:
#             allChapters.append(c)

# print(len(allChapters))

In [11]:
# import pandas as pd

# resDf = pd.DataFrame.from_dict(allChapters)
# # print(resDf)
# resDf.sort_values(by=[0],inplace=True)
# resDf.head()

In [12]:
# outputPath = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志'
# write_file_to = '{0}/{1}'.format(outputPath, 'allchapters_20190819_v2.xlsx')
# resDf.to_excel(write_file_to, index=True)

# test

In [13]:
# dupbooks = {}
# for key in bookTitles:
# #     print(bookTitles[key])
#     if len(bookTitles[key]) > 1:
# #         print(key, bookTitles[key])
#         dupbooks[key] = bookTitles[key]
# print(len(dupbooks))

In [14]:
# import pandas as pd

# resDf = pd.DataFrame.from_dict(dupbooks, orient='index')
# # print(resDf)
# resDf.sort_index(inplace=True)
# resDf.head()

# print(len(resDf))

In [15]:
# outputPath = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志'
# write_file_to = '{0}/{1}'.format(outputPath, 'gazetteers_20190811_v2.xlsx')
# resDf.to_excel(write_file_to, index=True)

In [16]:
# import pandas as pd

# resDf = pd.DataFrame(chapterTitles, columns=['section'])
# resDf.sort_values(by=['section'], inplace=True)
# resDf.head()

# print(len(resDf))

In [17]:
# outputPath = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志'
# write_file_to = '{0}/{1}'.format(outputPath, 'gazetteers_20190811_v1.xlsx')
# resDf.to_excel(write_file_to, index=False)

In [18]:
# count = 0
# for key in chapterTitles:
#     print(key, chapterTitles[key])
#     count += 1
#     if count > 5:
#         break

In [19]:
# # Remove bookname
# booknamecnt = 0
# for sec in chapterTitles:
# #     print(type(sec))
#     if sec == None:
#         continue
#     titles = chapterTitles[sec]
    
#     for tit in titles:
#         if tit == None:
#             continue
# #         print(sec)
#         if sec.startswith(tit):
#             if tit not in bookTitles:
#                 # remove the same section names in parent and sub section both
#                 if sec == tit:
#                     chapterTitles[sec].remove(tit)
#                     print(sec, tit)
#             else:
# #                 print(tit)
#                 chapterTitles[sec].remove(tit)

#             booknamecnt += 1
        
#         # remove parent section is book
# #         if tit in bookTitles:
# #             chapterTitles[sec].remove(tit)
# #             booknamecnt += 1
# #             print(sec, tit)
            
# print(booknamecnt)
# # print(len(chapterTitles))

In [20]:
# bookTitles['成安縣志']

In [21]:
# remove parent section is book
# for sec in chapterTitles:
# #     print(type(sec))
#     if sec == None:
#         continue
#     titles = chapterTitles[sec]
    
#     for tit in titles:
#         if tit == None:
#             continue

#         if tit in bookTitles:
#             # remove the parent section name which is the same as bookname
#             chapterTitles[sec].remove(tit)
            
# print(len(chapterTitles))

In [22]:
# 僊居水卷之一
# type(chapterTitles)

In [23]:
# import re

# for key in chapterTitles:
#     titles = chapterTitles[key]
#     for t in titles:
#         x = re.search(".*卷[首之上下一二三四五六七八九十]*", t)
#         if x != None:
#             print(t, x)
#             chapterTitles[key].remove(t)

In [24]:
# chapterTitles